In [30]:
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine
from IPython.display import display


In [38]:

load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_ENTRADA = os.getenv("DB_ENTRADA")
DB_SAI = os.getenv("DB_SAI")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

ENTRADA = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_ENTRADA}"
SAIDA = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_SAI}'

engine_entrada = create_engine(ENTRADA)
engine_saida = create_engine(SAIDA)


query = "SELECT * FROM endereco"
dados_originais_df = pd.read_sql(query, engine_entrada)
df_anonimizado = dados_originais_df.copy()


In [39]:
def generalizar_rua(rua):
    if not isinstance(rua, str):
        return "Rua desconhecida"
    tipo = rua.split()[0]  
    return f"{tipo} X"

df_anonimizado["rua"] = df_anonimizado["rua"].map(generalizar_rua)



In [40]:
def generalizar_cidade(cidade):
    if not isinstance(cidade, str):
        return "Cidade desconhecida"
    tipo = cidade.split()[0]  
    return f"Cidade X"

df_anonimizado["cidade"] = df_anonimizado["cidade"].map(generalizar_cidade)


In [41]:
def anonimizar_numero(numero, chave):
    numero = str(numero)
    chave = str(chave).lower()
    i = 0
    for l in numero:
        return "*****"

chave = 5

df_anonimizado["numero"] = df_anonimizado["numero"].apply(lambda x: anonimizar_numero(x, chave))



In [42]:
def abstracao_pais(pais):
    
    pais= pais.strip()

    uf = pais[:2].upper()
    return f"{uf}"

df_anonimizado["pais"] = df_anonimizado["pais"].apply(abstracao_pais)


In [43]:
bairro_para_regiao = {
    "Bela Vista": "Centro",
    "Vila Mariana": "Zona Sul",
    "Pinheiros": "Zona Oeste",
    "Vila Carrão": "Zona Leste",
    "Mooca": "Zona Leste"
}


df_anonimizado["bairro"] = df_anonimizado["bairro"].map(bairro_para_regiao)

df_anonimizado["bairro"] = df_anonimizado["bairro"].fillna("Região desconhecida")



In [44]:
def anonimizar_digitos(valor):
    valor_str = str(valor)
    valor_anonimizado = ''.join('*' if caractere.isdigit() else caractere for caractere in valor_str)
    return valor_anonimizado

df_anonimizado["cep"] = df_anonimizado["cep"].map(anonimizar_digitos)


In [46]:
from sqlalchemy import text

with engine_saida.begin() as conn:
    # por depender de outras tabelas, limpa a tabela sem quebrar FK e reinicia o ID
    conn.execute(text("TRUNCATE TABLE endereco RESTART IDENTITY CASCADE"))

df_anonimizado.to_sql(name='endereco', con=engine_saida, if_exists='append', index=False)



5

In [13]:
#mandar para o csv
dados_originais_df.to_csv('original_endereco.csv')
df_anonimizado.to_csv('pseudo_endereco.csv')